# Comparing the 2024 and 2025 Michelin Guide to France

In [14]:
import pandas as pd

In [15]:
france_24 = pd.read_csv('../../../2024/data/France/all_restaurants(arrondissements).csv')
france_25 = pd.read_csv('../../data/France/all_restaurants(arrondissements).csv')

In [16]:
france_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1017 non-null   object 
 1   address         1017 non-null   object 
 2   location        1017 non-null   object 
 3   arrondissement  1017 non-null   object 
 4   department_num  1017 non-null   object 
 5   department      1017 non-null   object 
 6   capital         1017 non-null   object 
 7   region          1017 non-null   object 
 8   price           1017 non-null   object 
 9   cuisine         1017 non-null   object 
 10  url             973 non-null    object 
 11  award           1017 non-null   object 
 12  stars           1017 non-null   float64
 13  longitude       1017 non-null   float64
 14  latitude        1017 non-null   float64
dtypes: float64(3), object(12)
memory usage: 119.3+ KB


In [17]:
france_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2991 entries, 0 to 2990
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            2991 non-null   object 
 1   address         2991 non-null   object 
 2   location        2991 non-null   object 
 3   arrondissement  2985 non-null   object 
 4   department_num  2991 non-null   object 
 5   department      2991 non-null   object 
 6   capital         2991 non-null   object 
 7   region          2991 non-null   object 
 8   price           2991 non-null   object 
 9   cuisine         2991 non-null   object 
 10  url             2840 non-null   object 
 11  award           2991 non-null   object 
 12  stars           2991 non-null   float64
 13  greenstar       2991 non-null   int64  
 14  longitude       2991 non-null   float64
 15  latitude        2991 non-null   float64
dtypes: float64(3), int64(1), object(12)
memory usage: 374.0+ KB


In [34]:
non_selected_25 = france_25[france_25['stars'] != 0.25]
print(len(non_selected_25))

983


----
&nbsp;
## Identify new or removed entries focusing on 1+ star restaurants

In [18]:
# Filter france_24 and france_25 for star ratings >= 1
subset_24 = france_24[france_24['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]
subset_25 = france_25[france_25['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]

# Turn the filtered data into sets of tuples
set_subset_24 = set(tuple(x) for x in subset_24.values)
set_subset_25 = set(tuple(x) for x in subset_25.values)

# Find new and closed Michelin stars
potential_new_stars = set_subset_25 - set_subset_24
potential_closed_stars = set_subset_24 - set_subset_25

In [19]:
print(f'Potentially entered the guide:\n{len(potential_new_stars)} starred restaurants in 2025 guide not in 2025')
print(f'\nPotentially left the guide:\n{len(potential_closed_stars)} starred restaurants in 2024 guide not in 2025')

Potentially entered the guide:
121 starred restaurants in 2025 guide not in 2025

Potentially left the guide:
144 starred restaurants in 2024 guide not in 2025


----
&nbsp;
### Using Pandas method
#### The above does not tell us how the distribution has changed.

In [20]:
# Now find name changes. For this, we need to find common addresses with different names
common_locations = set(subset_24['url']).intersection(set(subset_25['url']))

# Filtering out the common url from both DataFrames
common_24 = subset_24[subset_24['url'].isin(common_locations)]
common_25 = subset_25[subset_25['url'].isin(common_locations)]

In [21]:
# Merge the data frames on 'url' and 'location'
merged_df = pd.merge(common_24, common_25, on=['url', 'location'], suffixes=('_24', '_25'))

# Filter entries where names or star ratings have changed
name_star_changes = merged_df[(merged_df['name_24'] != merged_df['name_25']) | (merged_df['stars_24'] != merged_df['stars_25'])]

# Rename columns to reflect the data properly
name_star_changes = name_star_changes[['name_24', 'name_25', 'address_24', 'address_25', 'location', 'stars_24', 'stars_25']]

In [22]:
name_star_changes

,name_24,name_25,address_24,address_25,location,stars_24,stars_25
30,La Table des Amis by Christophe Bacquié,La Table des Amis,"Les Mas Les Eydins, 2420 chemin du Four","Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",2.0,2.0
33,L'Abysse au Pavillon Ledoyen,Pavyllon,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",2.0,1.0
100,Le Grand Contrôle,Ducasse au Château de Versailles - Le Grand Co...,12 rue de l'Indépendance-Américaine,12 rue de l'Indépendance-Américaine,"Versailles, 78000",1.0,1.0
126,Chakaiseiki Akiyoshi,Chakaiseki Akiyoshi,59 rue Letellier,59 rue Letellier,"Paris, 75015",1.0,1.0
220,Pavyllon,L'Abysse au Pavillon Ledoyen,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",1.0,2.0
251,Auberge St-Laurent,Auberge Saint-Laurent,1 rue de la Fontaine,1 rue de la Fontaine,"Sierentz, 68510",1.0,1.0
310,Hostellerie de Levernois,Table de Levernois,15 rue du Golf,"Hostellerie de Levernois, 15 rue du Golf","Levernois, 21200",1.0,1.0
513,Château de Collias,L'Hirondelle - Château de Collias,8 bis chemin du Barry,8 bis chemin du Barry,"Collias, 30210",1.0,1.0


We're focusing on the changes from ⭐️$\implies$⭐️⭐️ and from ⭐️⭐️$\implies$⭐️⭐️⭐️ (or significant demotions)

We assign a similarity score to the different names and change `france_23` in place

In [23]:
from fuzzywuzzy import fuzz

# Function to preprocess and compare names
def preprocess(text):
    # Basic preprocessing: lowercasing and stripping spaces
    return text.lower().strip()

def compare_names(name1, name2):
    name1 = preprocess(name1)
    name2 = preprocess(name2)
    return fuzz.token_sort_ratio(name1, name2)  # Comparing with sorted tokens to handle reordering

# Applying the comparison across the DataFrame rows
name_star_changes['similarity_score'] = name_star_changes.apply(lambda x: compare_names(x['name_24'], x['name_25']), axis=1)

In [24]:
name_star_changes

,name_24,name_25,address_24,address_25,location,stars_24,stars_25,similarity_score
30,La Table des Amis by Christophe Bacquié,La Table des Amis,"Les Mas Les Eydins, 2420 chemin du Four","Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",2.0,2.0,62
33,L'Abysse au Pavillon Ledoyen,Pavyllon,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",2.0,1.0,39
100,Le Grand Contrôle,Ducasse au Château de Versailles - Le Grand Co...,12 rue de l'Indépendance-Américaine,12 rue de l'Indépendance-Américaine,"Versailles, 78000",1.0,1.0,50
126,Chakaiseiki Akiyoshi,Chakaiseki Akiyoshi,59 rue Letellier,59 rue Letellier,"Paris, 75015",1.0,1.0,97
220,Pavyllon,L'Abysse au Pavillon Ledoyen,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",1.0,2.0,39
251,Auberge St-Laurent,Auberge Saint-Laurent,1 rue de la Fontaine,1 rue de la Fontaine,"Sierentz, 68510",1.0,1.0,92
310,Hostellerie de Levernois,Table de Levernois,15 rue du Golf,"Hostellerie de Levernois, 15 rue du Golf","Levernois, 21200",1.0,1.0,57
513,Château de Collias,L'Hirondelle - Château de Collias,8 bis chemin du Barry,8 bis chemin du Barry,"Collias, 30210",1.0,1.0,72


In [12]:
# Filter rows where the similarity score is greater than 65 (by inspection)
high_similarity_rows = name_star_changes[name_star_changes['similarity_score'] >= 65]

In [13]:
# Change 'Restaurant Marcon' manually (similarity score of 46)
france_23.loc[4, 'name'] = 'Restaurant Marcon'

In [14]:
# Iterate over these rows to update the names in france_23
for _, row in high_similarity_rows.iterrows():
    # Update france_23 where the url and location match
    france_23.loc[(france_23['address'] == row['address_23']) & (france_23['location'] == row['location']), 'name'] = row['name_24']

---
&nbsp;
### New ⭐⭐⭐ Restaurants, New ⭐⭐ Restaurants and demotions

In [25]:
# Identify New Two-Star and Three-Star Restaurants
# Filter france_24 and france_25 for star ratings >= 1
subset_24 = france_24[france_24['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]
subset_25 = france_25[france_25['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]

In [26]:
# Now find name changes. For this, we need to find common addresses with different names
common_locations = set(subset_24['name']).intersection(set(subset_25['name']))

# Filtering out the common url from both DataFrames
common_24 = subset_24[subset_24['name'].isin(common_locations)]
common_25 = subset_25[subset_25['name'].isin(common_locations)]

In [27]:
# Merge the data frames on 'name' and 'location'
merged_df = pd.merge(common_24, common_25, on=['name', 'location', 'url'], suffixes=('_24', '_25'))

In [29]:
# Filter entries where names or star ratings have changed
star_changes = merged_df[(merged_df['stars_24'] != merged_df['stars_25'])]

# Rename columns to reflect the data properly
star_changes = star_changes[['name', 'address_25', 'location', 'url', 'stars_24', 'stars_25']]

In [30]:
star_changes

,name,address_25,location,url,stars_24,stars_25


In [20]:
# We have one demotion from 2 star to 1 star to flag as this won't be picked up with analysis of 2 and 3 stars
star_changes.loc[51]

name                          Auberge du Cheval Blanc
address_24                       4 rue de Wissembourg
location                               Lembach, 67510
url           https://www.cheval-blanc-lembach.fr/fr/
stars_23                                          2.0
stars_24                                          1.0
Name: 51, dtype: object

We now need to find new entries at two star and three star

In [31]:
# Convert subset DataFrames to sets of tuples to identify unique restaurants
unique_24 = set(tuple(x) for x in subset_24[['name', 'location', 'stars']].values)
unique_25 = set(tuple(x) for x in subset_25[['name', 'location', 'stars']].values)

# Identify new restaurants in 2024 that were not in 2023
new_restaurants = unique_25 - unique_24

# Convert the set of new restaurants back to a DataFrame
df = pd.DataFrame(list(new_restaurants), columns=['name', 'location', 'stars'])

# Filter for restaurants that have two or three stars
new_high_stars = df[(df['stars'] == 2) | (df['stars'] == 3)]

new_high_stars = new_high_stars.sort_values(by=['stars'], ascending=[False])

In [32]:
new_high_stars

,name,location,stars
0,La Table des Amis,"Bonnieux, 84480",2.0
2,Cédric Burtin,"Saint-Rémy, 71100",2.0


It was announced in the press that there are two new 3* and eight new 2*.

In [23]:
from fuzzywuzzy import process

# Define a threshold for matching
threshold = 70

# Create a function to perform fuzzy matching
def get_matches(new_entry, old_data):
    # Use fuzzywuzzy's process.extract to get matches with a score
    matches = process.extract(new_entry['name'], old_data['name'], scorer=fuzz.token_set_ratio, limit=None)
    
    # Filter matches based on a threshold score
    high_score_matches = [match for match in matches if match[1] >= threshold]
    return high_score_matches

In [24]:
# Apply fuzzy matching to the new high-star restaurants
new_high_stars['matches_in_23'] = new_high_stars.apply(lambda x: get_matches(x, france_23), axis=1)

In [25]:
# Display results
new_high_stars[['name', 'location', 'stars', 'matches_in_23']]

,name,location,stars,matches_in_23
72,La Table du Castellet,"Le Castellet, 83330",3.0,"[(La Table, 100, 939), (La Tablée, 100, 1010),..."
81,Le Gabriel - La Réserve Paris,"Paris, 75008",3.0,"[(Le Gabriel - La Réserve Paris, 100, 54), (Le..."
9,Le Jules Verne,"Paris, 75007",2.0,"[(Le Jules Verne, 100, 348)]"
18,L'Orangerie,"Paris, 75008",2.0,"[(L'Orangerie, 100, 288), (L'Orangerie, 100, 2..."
31,René et Maxime Meilleur,"Saint-Martin-de-Belleville, 73440",2.0,"[(René et Maxime Meilleur, 100, 1), (Simple et..."
35,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",2.0,"[(Le Coquillage, 100, 46), (Le Cottage, 70, 849)]"
38,Maison Ronan Kervarrec,"Saint-Grégoire, 35760",2.0,"[(Maison Ronan Kervarrec, 100, 166)]"
45,Maison Benoît Vidal,"Annecy, 74940",2.0,"[(Vidal, 100, 638)]"
50,La Table des Amis by Christophe Bacquié,"Bonnieux, 84480",2.0,"[(La Table, 100, 939), (La Tablée, 100, 1010),..."
58,La Pyramide - Maison Henriroux,"Vienne, 38200",2.0,"[(La Pyramide - Patrick Henriroux, 86, 98), (L..."


In [26]:
# Create a function to perform fuzzy matching and return the best match above the threshold
def get_best_match(new_entry, old_data):
    # Use fuzzywuzzy's process.extractOne to get the best match with its score
    best_match, score, index = process.extractOne(new_entry['name'], old_data['name'], scorer=fuzz.token_set_ratio)
    
    # Check if the match score is above the threshold and return the match and corresponding star rating
    if score >= threshold:
        return (old_data.iloc[index]['name'], old_data.iloc[index]['location']), score, old_data.iloc[index]['stars']
    else:
        return None, None, None

In [27]:
# Apply fuzzy matching to the new high-star restaurants
new_high_stars[['best_match_23', 'match_score_23', 'stars_23']] = new_high_stars.apply(
    lambda x: get_best_match(x, france_23), axis=1, result_type='expand')

# Display results
new_high_stars[['name', 'location', 'stars', 'stars_23', 'best_match_23']]

,name,location,stars,stars_23,best_match_23
72,La Table du Castellet,"Le Castellet, 83330",3.0,0.5,"(La Table, Tourtour, 83690)"
81,Le Gabriel - La Réserve Paris,"Paris, 75008",3.0,2.0,"(Le Gabriel - La Réserve Paris, Paris, 75008)"
9,Le Jules Verne,"Paris, 75007",2.0,1.0,"(Le Jules Verne, Paris, 75007)"
18,L'Orangerie,"Paris, 75008",2.0,1.0,"(L'Orangerie, Paris, 75008)"
31,René et Maxime Meilleur,"Saint-Martin-de-Belleville, 73440",2.0,3.0,"(René et Maxime Meilleur, Saint-Martin-de-Bell..."
35,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",2.0,2.0,"(Le Coquillage, Cancale, 35350)"
38,Maison Ronan Kervarrec,"Saint-Grégoire, 35760",2.0,1.0,"(Maison Ronan Kervarrec, Saint-Grégoire, 35760)"
45,Maison Benoît Vidal,"Annecy, 74940",2.0,0.5,"(Vidal, Saint-Julien-Chapteuil, 43260)"
50,La Table des Amis by Christophe Bacquié,"Bonnieux, 84480",2.0,0.5,"(La Table, Tourtour, 83690)"
58,La Pyramide - Maison Henriroux,"Vienne, 38200",2.0,2.0,"(La Pyramide - Patrick Henriroux, Vienne, 38200)"


We define a function to return the status of the restaurant after inspection of the above df

In [28]:
# Define a function to determine the status of the restaurant
def determine_status(row):
    # Extract the postal code from the location and best_match_23
    postal_code = row['location'].split(', ')[-1]
    match_postal_code = row['best_match_23'][1].split(', ')[-1] if row['best_match_23'] else None

    # Check if the postal codes match
    if postal_code != match_postal_code:
        return 'new award'
    else:
        # Check for promotion or demotion
        if row['stars_23'] and row['stars']:
            if row['stars_23'] < row['stars']:
                return 'promoted'
            elif row['stars_23'] > row['stars']:
                return 'demoted'
        # If stars are the same or no match, return None (for removal)
        return None

In [29]:
# Apply the function to each row in the DataFrame
new_high_stars['status'] = new_high_stars.apply(determine_status, axis=1)

# update the 'stars_23' based on the status
new_high_stars.loc[new_high_stars['status'] == 'new award', 'stars_23'] = 'New Entry'

# Remove the rows where the status is None (meaning no change in stars)
new_high_stars = new_high_stars.dropna(subset=['status'])

# Display the updated DataFrame
major_star_changes = new_high_stars[['name', 'location', 'stars', 'stars_23', 'status']]

In [30]:
major_star_changes

,name,location,stars,stars_23,status
72,La Table du Castellet,"Le Castellet, 83330",3.0,New Entry,new award
81,Le Gabriel - La Réserve Paris,"Paris, 75008",3.0,2.0,promoted
9,Le Jules Verne,"Paris, 75007",2.0,1.0,promoted
18,L'Orangerie,"Paris, 75008",2.0,1.0,promoted
31,René et Maxime Meilleur,"Saint-Martin-de-Belleville, 73440",2.0,3.0,demoted
38,Maison Ronan Kervarrec,"Saint-Grégoire, 35760",2.0,1.0,promoted
45,Maison Benoît Vidal,"Annecy, 74940",2.0,New Entry,new award
50,La Table des Amis by Christophe Bacquié,"Bonnieux, 84480",2.0,New Entry,new award
61,Maison Ruggieri,"Paris, 75008",2.0,1.0,promoted
79,Sylvestre Wahid - Les Grandes Alpes,"Courchevel, 73120",2.0,1.0,promoted


In [31]:
# We manually write the demotion of Auberge du Cheval Blanc to star_changes
aub_chev = star_changes.loc[51]
aub_chev

name                          Auberge du Cheval Blanc
address_24                       4 rue de Wissembourg
location                               Lembach, 67510
url           https://www.cheval-blanc-lembach.fr/fr/
stars_23                                          2.0
stars_24                                          1.0
Name: 51, dtype: object

In [32]:
data_to_append = {
    'name': aub_chev['name'],
    'location': aub_chev['location'],
    'stars': aub_chev['stars_24'],
    'stars_23': aub_chev['stars_23'],
    'status': 'demoted'
}

row_to_append = pd.DataFrame([data_to_append])
major_star_changes = pd.concat([major_star_changes, row_to_append], ignore_index=True)

In [33]:
major_star_changes

,name,location,stars,stars_23,status
0,La Table du Castellet,"Le Castellet, 83330",3.0,New Entry,new award
1,Le Gabriel - La Réserve Paris,"Paris, 75008",3.0,2.0,promoted
2,Le Jules Verne,"Paris, 75007",2.0,1.0,promoted
3,L'Orangerie,"Paris, 75008",2.0,1.0,promoted
4,René et Maxime Meilleur,"Saint-Martin-de-Belleville, 73440",2.0,3.0,demoted
5,Maison Ronan Kervarrec,"Saint-Grégoire, 35760",2.0,1.0,promoted
6,Maison Benoît Vidal,"Annecy, 74940",2.0,New Entry,new award
7,La Table des Amis by Christophe Bacquié,"Bonnieux, 84480",2.0,New Entry,new award
8,Maison Ruggieri,"Paris, 75008",2.0,1.0,promoted
9,Sylvestre Wahid - Les Grandes Alpes,"Courchevel, 73120",2.0,1.0,promoted


----
&nbsp;
## Merging the promotions/demotions with `france_24`

In [34]:
france_24.head()

,name,address,location,arrondissement,department_num,department,capital,region,price,cuisine,url,award,stars,longitude,latitude
0,La Table du Castellet,"3001 route des Hauts-du-Camp, au Circuit Paul ...","Le Castellet, 83330",Toulon,83,Var,Toulon,Provence-Alpes-Côte d'Azur,€€€€,Creative,http://www.hotelducastellet.net/fr/restaurants...,3 Stars,3.0,5.783887,43.249929
1,Plénitude - Cheval Blanc Paris,8 quai du Louvre,"Paris, 75001",1st (Louvre),75,Paris,Paris,Île-de-France,€€€€,Creative,https://www.chevalblanc.com/fr/maison/paris/,3 Stars,3.0,2.342159,48.858815
2,Le Petit Nice,Anse de Maldormé,"Marseille, 13007",Marseille,13,Bouches-du-Rhône,Marseille,Provence-Alpes-Côte d'Azur,€€€€,Seafood,https://www.passedat.fr,3 Stars,3.0,5.352121,43.280086
3,Mirazur,30 avenue Aristide-Briand,"Menton, 06500",Nice,06,Alpes-Maritimes,Nice,Provence-Alpes-Côte d'Azur,€€€€,Creative,https://www.mirazur.fr/,3 Stars,3.0,7.528051,43.785930
4,AM par Alexandre Mazzia,9 rue François-Rocca,"Marseille, 13008",Marseille,13,Bouches-du-Rhône,Marseille,Provence-Alpes-Côte d'Azur,€€€€,Creative,https://www.alexandre-mazzia.com/,3 Stars,3.0,5.386233,43.270110


In [35]:
# Merge major_star_changes with france_24 on 'name' and 'location'
merged_changes = pd.merge(major_star_changes, france_24, on=['name', 'location'], how='left')

merged_changes.rename(columns={'stars_y': 'stars'}, inplace=True)
column_order = ['name', 'address', 'location', 'arrondissement', 'department_num', 'department', 'capital', 'region',
       'price', 'cuisine', 'url', 'award', 'stars', 'stars_23', 'status', 'longitude', 'latitude']
merged_changes = merged_changes[column_order]

In [36]:
merged_changes

,name,address,location,arrondissement,department_num,department,capital,region,price,cuisine,url,award,stars,stars_23,status,longitude,latitude
0,La Table du Castellet,"3001 route des Hauts-du-Camp, au Circuit Paul ...","Le Castellet, 83330",Toulon,83,Var,Toulon,Provence-Alpes-Côte d'Azur,€€€€,Creative,http://www.hotelducastellet.net/fr/restaurants...,3 Stars,3.0,New Entry,new award,5.783887,43.249929
1,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,Creative,https://www.lareserve-paris.com/,3 Stars,3.0,2.0,promoted,2.313208,48.869731
2,Le Jules Verne,Tour Eiffel - Avenue Gustave-Eiffel,"Paris, 75007",7th (Palais-Bourbon),75,Paris,Paris,Île-de-France,€€€€,Modern Cuisine,https://www.restaurants-toureiffel.com/fr/rest...,2 Stars,2.0,1.0,promoted,2.294532,48.858232
3,L'Orangerie,"Four Seasons George V, 31 avenue George-V","Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,Modern Cuisine,https://www.fourseasons.com/fr/paris/dining/re...,2 Stars,2.0,1.0,promoted,2.300826,48.868785
4,René et Maxime Meilleur,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440",Albertville,73,Savoie,Chambéry,Auvergne-Rhône-Alpes,€€€€,"Creative, Regional Cuisine",https://www.la-bouitte.com/fr/,2 Stars,2.0,3.0,demoted,6.513306,45.369046
5,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760",Rennes,35,Ille-et-Vilaine,Rennes,Brittany,€€€€,Modern Cuisine,http://www.le-saison.com/fr,2 Stars,2.0,1.0,promoted,-1.680556,48.153301
6,Maison Benoît Vidal,"Sur-les-Bois, 79 route de Thônes","Annecy, 74940",Annecy,74,Haute-Savoie,Annecy,Auvergne-Rhône-Alpes,€€€€,Creative,https://www.maison-benoit-vidal.com,2 Stars,2.0,New Entry,new award,6.171891,45.922698
7,La Table des Amis by Christophe Bacquié,"Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",Apt,84,Vaucluse,Avignon,Provence-Alpes-Côte d'Azur,€€€€,Modern Cuisine,http://www.leseydins.com,2 Stars,2.0,New Entry,new award,5.301851,43.851187
8,Maison Ruggieri,11 rue Treilhard,"Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,"Modern Cuisine, Creative",http://www.maisonruggieri.fr,2 Stars,2.0,1.0,promoted,2.314134,48.877027
9,Sylvestre Wahid - Les Grandes Alpes,"28 rue de l'Église, Courchevel 1850","Courchevel, 73120",Albertville,73,Savoie,Chambéry,Auvergne-Rhône-Alpes,€€€€,Creative,https://restaurantsylvestre.com,2 Stars,2.0,1.0,promoted,6.634931,45.414421


----
&nbsp;
## Changes in the michelin guide

In [45]:
stars_23 = france_23[france_23['stars'] >= 1]
stars_24 = france_24[france_24['stars'] >= 1]

In [49]:
# Count the occurrences of each star in france_23 and france_24
stars_count_23 = stars_23.groupby('stars').size()
stars_count_24 = stars_24.groupby('stars').size()

# Ensure all possible star values are represented, even if there are no restaurants with that count
star_ratings = [1, 2, 3]
stars_count_23 = stars_count_23.reindex(star_ratings)
stars_count_24 = stars_count_24.reindex(star_ratings)

# Create a DataFrame to display the counts side by side
star_comparison_table = pd.DataFrame({
    'Star Rating': star_ratings,
    '2023 Guide': stars_count_23.values,
    '2024 Guide': stars_count_24.values
})

# Display the table
star_comparison_table4

,Star Rating,2023 Guide,2024 Guide
0,1,520,530
1,2,72,73
2,3,28,29


----
&nbsp;
## New restaurants by region

In [37]:
# Function import
from Functions.functions_visualisation import top_restaurants

In [38]:
top_restaurants(merged_changes, granularity='region', star_rating=3, top_n=5)

Only 2 unique regions found.

Top 2 regions with most ⭐⭐⭐ restaurants:


Region: Provence-Alpes-Côte d'Azur
1 ⭐⭐⭐ Restaurant



Region: Île-de-France
1 ⭐⭐⭐ Restaurant


In [40]:
top_restaurants(merged_changes, granularity='region', star_rating=2, top_n=5)

Only 4 unique regions found.

Top 4 regions with most ⭐⭐ restaurants:


Region: Auvergne-Rhône-Alpes
3 ⭐⭐ Restaurants



Region: Île-de-France
3 ⭐⭐ Restaurants





Region: Brittany
1 ⭐⭐ Restaurant





Region: Provence-Alpes-Côte d'Azur
1 ⭐⭐ Restaurant




----
&nbsp;
## Postscript - Useful code
Searching for particular restaurants - Verify the differences

In [143]:
# Function to find potential matches based on the restaurant name
def find_potential_matches(search_name, df, threshold=70):
    # Use fuzzy matching to find potential matches and their scores
    potential_matches = process.extract(search_name, df['name'], scorer=fuzz.token_set_ratio)
    
    # Filter out matches that meet or exceed the threshold score
    good_matches = [(match[0], match[1]) for match in potential_matches if match[1] >= threshold]
    return good_matches

In [149]:
# The name of the restaurant you're interested in
search_name = "La Table du Castellet"

In [150]:
# Use the function to find matches in france_23
matches_in_23 = find_potential_matches(search_name, france_23, threshold)

# Use the function to find matches in france_24
matches_in_24 = find_potential_matches(search_name, france_24, threshold)

# Display the results
print("Potential matches in 2023 data:")
for match in matches_in_23:
    print(f"Name: {match[0]}, Score: {match[1]}")

print("\nPotential matches in 2024 data:")
for match in matches_in_24:
    print(f"Name: {match[0]}, Score: {match[1]}")

Potential matches in 2023 data:
Name: La Table, Score: 100
Name: La Tablée, Score: 100
Name: La Table du 11, Score: 88
Name: La Table d'Asten, Score: 81
Name: La Table d'Auzeville, Score: 78

Potential matches in 2024 data:
Name: La Table du Castellet, Score: 100
Name: La Table, Score: 100
Name: La Table du 11, Score: 88
Name: La Table d'Asten, Score: 81
Name: La Table du Square, Score: 76
